In [1]:
import re
import numpy as np
import pickle
import itertools
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
def remove_brackets(var):
    #return re.sub("[\(].*?[\)]", "", var)
    #including year in movie
    i = var.find(")")
    
    if i == -1:
        return re.sub("[\(].*?[\)]", "", var)
    
    else:
        return var[:i+1] + re.sub("[\(].*?[\)]", "", var[i+1:])

def clean_actor_name(name):
    name = name.replace("&#378;","z")
    return name
    

#def total_length(list_of_lists):
#    return len(set(itertools.chain.from_iterable(([i[1:] for i in list_of_lists]))))

def total_length(dictionary):
    unique = set([])
    for k,v in dictionary.items():
        for i in v:
            unique.add(i)
    return len(unique)

In [3]:
def preprocess(actors):
    
    actor_movie_map = {}

    for act in actors:
        
        movies = act.split('\t\t')
        movie_names = map(remove_brackets, movies[1:])
        mov = list(set([movie.strip('\n').strip() for movie in movie_names]))
        
        if len(mov) >= 10:    
            actor_name =  clean_actor_name(movies[0])
            actor_movie_map[actor_name] = mov
            
    return actor_movie_map

In [4]:
actors = list(open("actor_movies.txt", "r"))
actress = list(open("actress_movies.txt", "r"))

In [5]:
actors_movie_map = preprocess(actors)
actress_movie_map = preprocess(actress)
actor_movie_map = {**actors_movie_map, **actress_movie_map}

In [6]:
movie_actor_map = {}

for actor, movies in actor_movie_map.items():
    
    for movie in movies:
        
        if movie_actor_map.get(movie):
            movie_actor_map[movie].append(actor)
        else:
            movie_actor_map[movie] = [actor]

In [7]:
mov_ratings = list(open("movie_ratings2.txt", "r"))

In [8]:
movie_rating_map = {}
for movie in movie_actor_map.keys():
    movie_rating_map[movie]=0
for mr in mov_ratings:
    mr = mr.strip('\n')
    a = mr.split('\t')
    movie_rating_map[a[0]]=float(a[1])

In [9]:
with open('actor_movie_map.pickle', 'wb') as handle:
    pickle.dump(actor_movie_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
rated_mov = [mr.split('\t')[0] for mr in mov_ratings]
rated_movies = list(set(movie_actor_map.keys()).intersection(set(rated_mov)))
rated_movie_actors = [movie_actor_map[mov][:] for mov in rated_movies]
rated_movie_actors = [item for sublist in rated_movie_actors for item in sublist]

In [11]:
actor_rating_map = {}
for actor in actor_movie_map.keys():
    actor_rating_map[actor] = 0
for actor in rated_movie_actors:
    mov_list = []
    for movie in actor_movie_map[actor]:
        mov_list.append(movie_rating_map[movie])
    avg_rating = np.mean(np.asarray(mov_list)[np.asarray(mov_list)>0])
    actor_rating_map[actor] = avg_rating

In [12]:
def predict_rating(movie):
    actors = movie_actor_map[movie]
    ratings = [actor_rating_map[actor] for actor in actors]
    avg_rating = np.mean(np.asarray(ratings)[np.asarray(ratings)>0])
    return(avg_rating)

In [13]:
predict_rating('Batman v Superman: Dawn of Justice (2016)')

6.496376711498748

In [14]:
predict_rating('Mission: Impossible - Rogue Nation (2015)')

6.4907514869503355

In [15]:
predict_rating('Minions (2015)')

6.8724113193421354

In [16]:
test_movies = ['Batman v Superman: Dawn of Justice (2016)','Mission: Impossible - Rogue Nation (2015)','Minions (2015)']
yhat = np.array([predict_rating(movie) for movie in test_movies])
yts = np.array([6.6,7.4,6.4])
print('The RMSE is',np.sqrt(np.sum((yts-yhat)**2)))

The RMSE is 1.0298752832481304


# Question 12

In [17]:
def features(movie_set):
    num_actors = [len(movie_actor_map[movie]) for movie in movie_set]
    movies_of_actors = []
    avg_rating = []
    for movie in movie_set:
        num_movies=0
        ratings = []
        for actor in movie_actor_map[movie]:
            num_movies+= len(actor_movie_map[actor])
            ratings.append(actor_rating_map[actor])
        movies_of_actors.append(num_movies)
        avg_rating.append(np.mean(ratings))
        
    return(num_actors,movies_of_actors,avg_rating)

In [18]:
test_movies = ['Batman v Superman: Dawn of Justice (2016)','Mission: Impossible - Rogue Nation (2015)','Minions (2015)']
Xtr = np.column_stack((features(rated_movies)))
Xts = np.column_stack((features(test_movies)))
ytr = np.array([movie_rating_map[movie] for movie in rated_movies])
yts = np.array([6.6,7.4,6.4])

In [19]:
sc = StandardScaler()
Xtr_scale = sc.fit_transform(Xtr)
Xts_scale = sc.transform(Xts)

In [20]:
model = RandomForestRegressor(max_depth=3, random_state=0)
model.fit(Xtr_scale,ytr)
yhat = model.predict(Xts_scale)
print('The RMSE is',np.sqrt(np.sum((yts-yhat)**2)))

The RMSE is 0.8273250120002603


In [21]:
model = LinearRegression()
model.fit(Xtr_scale,ytr)
yhat = model.predict(Xts_scale)
print('The RMSE is',np.sqrt(np.sum((yts-yhat)**2)))

The RMSE is 1.1929268555118546


# Without average rating feature

In [22]:
def features(movie_set):
    num_actors = [len(movie_actor_map[movie]) for movie in movie_set]
    movies_of_actors = []
    avg_rating = []
    for movie in movie_set:
        num_movies=0
        ratings = []
        for actor in movie_actor_map[movie]:
            num_movies+= len(actor_movie_map[actor])
            ratings.append(actor_rating_map[actor])
        movies_of_actors.append(num_movies)
        avg_rating.append(np.mean(ratings))
        
    return(num_actors,movies_of_actors)

In [23]:
test_movies = ['Batman v Superman: Dawn of Justice (2016)','Mission: Impossible - Rogue Nation (2015)','Minions (2015)']
Xtr = np.column_stack((features(rated_movies)))
Xts = np.column_stack((features(test_movies)))
ytr = np.array([movie_rating_map[movie] for movie in rated_movies])
yts = np.array([6.6,7.4,6.4])

In [24]:
sc = StandardScaler()
Xtr_scale = sc.fit_transform(Xtr)
Xts_scale = sc.transform(Xts)

C:\Users\madhu.kolli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [25]:
model = RandomForestRegressor(max_depth=3, random_state=0)
model.fit(Xtr_scale,ytr)
yhat = model.predict(Xts_scale)
print('The RMSE is',np.sqrt(np.sum((yts-yhat)**2)))

The RMSE is 0.7826507612237003


In [26]:
model = LinearRegression()
model.fit(Xtr_scale,ytr)
yhat = model.predict(Xts_scale)
print('The RMSE is',np.sqrt(np.sum((yts-yhat)**2)))

The RMSE is 1.7152775771773474
